In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from skimage.segmentation import mark_boundaries
from lime import lime_image
import cv2
import tensorflow as tf

# Define class names
class_names = ['Covid', 'Normal', 'Viral Pneumonia']

# Initialize LIME explainer
explainer = lime_image.LimeImageExplainer()

def grad_cam(input_model, image, cls, last_conv_layer_name):
    last_conv_layer = input_model.get_layer(last_conv_layer_name)
    iterate = tf.keras.models.Model([input_model.inputs], [input_model.output, last_conv_layer.output])

    with tf.GradientTape() as tape:
        inputs = tf.cast(image, tf.float32)
        tape.watch(inputs)
        predictions, conv_outputs = iterate(inputs)
        y_c = predictions[:, cls]

    grads = tape.gradient(y_c, conv_outputs)

    output, grads_val = conv_outputs[0], grads[0]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    return heatmap

def calculate_impression_score(lime_output, gradcam_output):
    # Resize the LIME output to match the GradCAM output dimensions
    lime_output_resized = cv2.resize(lime_output, (gradcam_output.shape[1], gradcam_output.shape[0]))
    # Calculate the similarity between Lime and Grad-CAM
    similarity = np.corrcoef(lime_output_resized.flatten(), gradcam_output.flatten())[0, 1]
    return similarity

# Loop over the first 3 images in the test dataset
lime_outputs = []
gradcam_outputs = []
explainability_acceptances = []

for i in range(3):
    # Select an example image from the test dataset
    img_path = training_set.filepaths[i+5]

    # Preprocess the selected image for VGG16
    processed_image = image.load_img(img_path, target_size=(224, 224))
    processed_image = image.img_to_array(processed_image)
    processed_image = np.expand_dims(processed_image, axis=0)
    processed_image = preprocess_input(processed_image)

    # Get the model's predictions
    preds = model.predict(processed_image)
    class_idx = np.argmax(preds[0])

    # Get the GradCAM heatmap
    gradcam_heatmap = grad_cam(model, processed_image, class_idx, 'block5_conv3')

    # Explain the model's predictions for the selected image using LIME
    explanation = explainer.explain_instance(processed_image[0], model.predict, top_labels=3, hide_color=0, num_samples=2000)
    lime_output, _ = explanation.get_image_and_mask(class_idx, positive_only=True, num_features=10, hide_rest=False)

    # Convert LIME output to grayscale for comparison
    lime_output_gray = cv2.cvtColor(lime_output, cv2.COLOR_BGR2GRAY)

    # Calculate the impression score
    impression_score = calculate_impression_score(lime_output_gray, gradcam_heatmap)

    # Calculate the explainability acceptance
    explainability_distance = np.sqrt(np.sum((cv2.resize(lime_output_gray, gradcam_heatmap.shape) - gradcam_heatmap) ** 2))
    explainability_acceptance = 1 - explainability_distance

    # Store expert outputs and acceptances
    lime_outputs.append(lime_output_gray)
    gradcam_outputs.append(gradcam_heatmap)
    explainability_acceptances.append(explainability_acceptance)

# Calculate Trustworthy Explainability Acceptance metric
TwA = np.mean(explainability_acceptances)

# Calculate standard error of Trustworthy Explainability Acceptance
SETwA = np.sqrt(np.mean((TwA - explainability_acceptances) ** 2))

# Calculate confidence of Trustworthy Explainability Acceptance
cTwA = 1 - 2 * SETwA

# Calculate system-level Trustworthy Explainability Acceptance
SystemTwA = np.mean(explainability_acceptances)

# Calculate standard error of system-level Trustworthy Explainability Acceptance
SESystem = np.sqrt(np.mean(SETwA ** 2))

# Print the calculated metrics
print("Trustworthy Explainability Acceptance (TwA):", TwA)
print("Standard Error of TwA (SETwA):", SETwA)
print("Confidence of TwA (cTwA):", cTwA)
print("System-level TwA:", SystemTwA)
print("Standard Error of System TwA (SESystem):", SESystem)



1/1 [==============================] - 0s 32ms/step


  0%|          | 0/2000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 31ms/step


  0%|          | 0/2000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step


  0%|          | 0/2000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step
Trustworthy Explainability Acceptance (TwA): -16411.637369791668
Standard Error of TwA (SETwA): 623.3876362999082
Confidence of TwA (cTwA): -1245.7752725998164
System-level TwA: -16411.637369791668
Standard Error of System TwA (SESystem): 623.3876362999082
